
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [15]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


In [16]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

FileNotFoundError: [Errno 2] No such file or directory: './content'

In [ ]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

In [ ]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-26 11:46:16.278655: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 11:46:16.333921: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 11:46:16.333983: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 11:46:16.336043: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 11:46:16.345343: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 11:46:16.348713: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [ ]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [ ]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [ ]:
tester.Loop(sort_by='r2',
            boundarie=-0.8,
            hidden_sizes = [[21, 10], [40, 15]],
            regularizers=[ 0.1,0.02],
            learning_rate=[ 0.1,0.01])

Testando combinacao1: Hidden Size=[15, 7], regularizer=0.1, learning_rate=0.1
+++++++++++ [1_0] | 1 ++++++++++++++++++


Elapsed time:  1.1219763279987092
Stopped at epoch:  12
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.48358453800028656
Stopped at epoch:  11
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.24712811700010207
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.29941353100002743
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.32678624500113074
Stopped at epoch:  9
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.2707069549996959
Stopped at epoch:  7
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Elapsed time:  0.21363656199901015
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/ste

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_0_6,-0.709229,-2.157226,-7.756551,-2.744279,1.891612,6.043070,6.078126,6.059568
model_1_0_5,-0.206329,-1.060760,-5.489429,-1.600145,1.335050,3.944386,4.504464,4.207953
model_1_0_4,-0.153314,-0.964854,-5.084094,-1.458281,1.276379,3.760818,4.223112,3.978368
model_1_0_3,-0.035820,-0.955581,-3.373060,-1.107122,1.146348,3.743069,3.035443,3.410069
model_1_0_2,-0.005212,-0.928365,-2.774131,-0.969193,1.112473,3.690976,2.619713,3.186852
model_1_0_0,0.079954,-0.342214,-3.066084,-0.661109,1.018219,2.569058,2.822364,2.688262
model_1_0_1,0.315291,0.060778,-2.013006,-0.196227,0.757771,1.797714,2.091398,1.935918


DataFrame salvo em ../results/metrics_1_0
+++++++++++ [1_1] | 1 ++++++++++++++++++
Elapsed time:  0.29550921999907587
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_1] | 2 ++++++++++++++++++
Elapsed time:  0.3391300629991747
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_1] | 3 ++++++++++++++++++
Elapsed time:  0.22544439699959185
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_1] | 4 ++++++++++++++++++
Elapsed time:  0.3593174569996336
Stopped at epoch:  9
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_1] | 5 ++++++++++++++++++
Elapsed time:  0.25569394800004375
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_1] | 6 ++++++++++++++++++
Elapsed time:  0.2968009329997585
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_1] | 7 ++++++++++++++++++
Elapsed time:  0.433668348

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_1_0,-0.103475,0.850790,-0.224306,0.512270,1.221221,0.123725,0.641315,0.367296
model_1_1_2,0.039594,0.236692,-7.630446,-2.269952,1.062886,0.632935,4.520789,2.462514
model_1_1_1,0.137169,0.827410,-1.179087,0.186113,0.954900,0.143112,1.141447,0.612916
model_1_1_9,0.224753,-0.192839,-8.233167,-2.717626,0.857970,0.989102,4.836506,2.799645
model_1_1_8,0.292569,-0.004559,-7.500454,-2.368033,0.782918,0.832980,4.452697,2.536376
model_1_1_7,0.357594,0.075659,-6.667341,-2.048572,0.710955,0.766464,4.016297,2.295799
model_1_1_5,0.380120,0.697078,-7.267409,-1.882746,0.686024,0.251183,4.330624,2.170919
model_1_1_6,0.384955,0.129192,-6.338205,-1.909628,0.680674,0.722074,3.843889,2.191163
model_1_1_10,0.435662,-0.194497,-5.123340,-1.700656,0.624556,0.990476,3.207520,2.033792
model_1_1_11,0.436961,-0.194265,-5.104562,-1.694371,0.623118,0.990284,3.197684,2.029058


...

DataFrame salvo em ../results/metrics_2_6
+++++++++++ [2_7] | 1 ++++++++++++++++++
Elapsed time:  0.41117646699967736
Stopped at epoch:  10
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_7] | 2 ++++++++++++++++++
Elapsed time:  0.2915023499990639
Stopped at epoch:  7
1/1 [==============================] - 0s 35ms/step
+++++++++++ [2_7] | 3 ++++++++++++++++++
Elapsed time:  1.5190419760001532
Stopped at epoch:  20
1/1 [==============================] - 0s 25ms/step
+++++++++++ [2_7] | 4 ++++++++++++++++++
Elapsed time:  0.28360596999846166
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [2_7] | 5 ++++++++++++++++++
Elapsed time:  0.32718959100020584
Stopped at epoch:  7
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_7] | 6 ++++++++++++++++++
Elapsed time:  0.3574368810004671
Stopped at epoch:  9
1/1 [==============================] - 0s 14ms/step
+++++++++++ [2_7] | 7 ++++++++++++++++++
Elapsed time:  0.7614432

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_8_0,-0.068908,-0.626081,-14.196223,-1.034853,1.182966,2.561800,1.324327,1.979460
model_2_8_1,0.161327,-1.753781,-7.799555,-1.732070,0.928164,4.338428,0.766867,2.657697
model_2_8_2,0.324717,-1.607761,-5.219084,-1.498179,0.747339,4.108383,0.541983,2.430173
model_2_8_3,0.411948,-1.256322,-5.110896,-1.192148,0.650800,3.554710,0.532555,2.132473
model_2_8_4,0.470785,-1.014117,-4.981794,-0.979126,0.585685,3.173129,0.521304,1.925250
model_2_8_5,0.503662,-0.883770,-4.766355,-0.858278,0.549300,2.967775,0.502529,1.807691
model_2_8_6,0.512501,-0.851407,-4.651750,-0.825727,0.539517,2.916788,0.492541,1.776027
model_2_8_7,0.515885,-0.834922,-4.689830,-0.813106,0.535773,2.890817,0.495860,1.763749
model_2_8_8,0.518242,-0.824615,-4.689433,-0.804282,0.533164,2.874579,0.495825,1.755165
model_2_8_9,0.518296,-0.824495,-4.687094,-0.804081,0.533105,2.874390,0.495621,1.754970


DataFrame salvo em ../results/metrics_2_8
+++++++++++ [2_9] | 1 ++++++++++++++++++
Elapsed time:  0.03561318200081587
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_9] | 2 ++++++++++++++++++
Elapsed time:  0.03021722399898863
Stopped at epoch:  0
1/1 [==============================] - 0s 23ms/step
+++++++++++ [2_9] | 3 ++++++++++++++++++
Elapsed time:  0.03477257099984854
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [2_9] | 4 ++++++++++++++++++
Elapsed time:  0.02743281499897421
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_9] | 5 ++++++++++++++++++
Elapsed time:  0.029691296000237344
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_9] | 6 ++++++++++++++++++
Elapsed time:  0.028418324000085704
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [2_9] | 7 ++++++++++++++++++
Elapsed time:  0.0281

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_9_0,0.527896,0.691622,0.570115,0.623179,0.52248,0.149233,0.3049,0.222489
model_2_9_22,0.527896,0.691622,0.570115,0.623179,0.52248,0.149233,0.3049,0.222489
model_2_9_21,0.527896,0.691622,0.570115,0.623179,0.52248,0.149233,0.3049,0.222489
model_2_9_20,0.527896,0.691622,0.570115,0.623179,0.52248,0.149233,0.3049,0.222489
model_2_9_19,0.527896,0.691622,0.570115,0.623179,0.52248,0.149233,0.3049,0.222489
model_2_9_18,0.527896,0.691622,0.570115,0.623179,0.52248,0.149233,0.3049,0.222489
model_2_9_17,0.527896,0.691622,0.570115,0.623179,0.52248,0.149233,0.3049,0.222489
model_2_9_16,0.527896,0.691622,0.570115,0.623179,0.52248,0.149233,0.3049,0.222489
model_2_9_15,0.527896,0.691622,0.570115,0.623179,0.52248,0.149233,0.3049,0.222489
model_2_9_14,0.527896,0.691622,0.570115,0.623179,0.52248,0.149233,0.3049,0.222489


DataFrame salvo em ../results/metrics_2_9
